In [13]:
import logging

from datetime import datetime
from csv import reader
from pathlib import Path

In [ ]:
class Template:
  pass

In [2]:
class Employee:
  """ Creates an employee object with additional functions."""
  def __init__(self, name: str, rank: int, position: str, hours: dict, lunch_hours: dict):
    self.name = name
    self.first_name = name.split()[0]
    self.initials = "".join(i[0] for i in name.split())

    self.rank = rank
    self.position = position

    self.hours = self._to_datetime(hours)
    self.phours = self._to_ptime(hours)
    self.lunch_hours = self._to_datetime(lunch_hours)
    self.plunch_hours = self._to_ptime(lunch_hours)

    self.programs = None

  def _to_datetime(self, hours):
    result = {}
    for day, times in hours.items():
      result[day] = [datetime.strptime(t.zfill(4), "%H%M").time() for t in times]
    return result
  
  def _to_ptime(self, hours):
    result = {}
    for day, times in hours.items():
      dt = [datetime.strptime(t.zfill(4), "%H%M") for t in times]
      hours = [str(t.hour % 12) or "12" for t in dt]
      mins = ["" if t.minute == 0 else str(t.minute) for t in dt]
      ft = [":".join([str(h), f"{m}"]).rstrip(":") for h, m in zip(hours, mins)]
      result[day] = ft
    return result

In [40]:
class Loader:
  """ Loads and converts employees to `Employee` objects and templates as `Template` objects"""
  def __init__(self):
    self.employees = []
    self.templates = []

    self.key = None
  
  def load_employees(self, employees_path: str, employees_key: str = ""):
    try:
      key_map = {}
      with open(employees_key, "r", encoding="utf-8") as keyfile:
        for line in keyfile:
          encoded, decoded = line.split("= ")
          key_map[encoded.strip()] = decoded.strip()
          self.key = key_map
    except FileNotFoundError as e:
      logging.exception(f"Path: '{employees_key}' does not exist. {e}")

    try:
      fields = []
      rows = []
      with open(employees_path, "r", encoding="utf-8") as csvfile:
        csvreader = reader(csvfile)
        fields = next(csvreader)
        for row in csvreader:
          rows.append(row)
      for row in rows:
        name = self.key.get(row[0], row[0]) if self.key else row[0]
        print(name)
    except FileNotFoundError as e:
      logging.exception(f"Path: '{employees_path}' does not exist. {e}")

In [41]:
loader = Loader()
loader.load_employees("../models/employees.csv", "../models/employees_key.txt")

Michelle DuPre
Rod Franco
Lea Stapleton
Anthony Young
Yami Ellis
Sonaite Debebe-Kumssa
Lindsey Taunton
Jessica Bryant
Chris Wright
Cat Alva
Shawn Sebro
Wil Meade
Janet Henson
Emaleigh Kitchen
Wendy Jones
Deborah Lancaster
Cheryl Campbell
Sydnee Matthews
Judy Cato
Jerry Conley
Shallie Church
Louie Jamison
Marion Mack


In [ ]:
chris = Employee("Chris Wright", 4, "full time", {"monday": ["1400", "2000"], "tuesday": ["900", "1730"]}, {"monday": ["1730", "1800"], "tuesday": ["1300", "1400"]})

# print(chris.first_name)
# print(chris.initials)
# print(chris.hours)
# print(chris.phours)
# print(chris.lunch_hours)
# print(chris.plunch_hours)

Chris
CW
{'monday': [datetime.time(14, 0), datetime.time(20, 0)], 'tuesday': [datetime.time(9, 0), datetime.time(17, 30)]}
{'monday': ['2', '8'], 'tuesday': ['9', '5:30']}
{'monday': [datetime.time(17, 30), datetime.time(18, 0)], 'tuesday': [datetime.time(13, 0), datetime.time(14, 0)]}
{'monday': ['5:30', '6'], 'tuesday': ['1', '2']}
